# Symbology mapping using dedicated service - Python

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** instrument's identifiers and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Symbology mapping sample is designed to help you searching and requesting instrument's specification data by using a wide range of commonly used fields.
* It takes instrument specification parameter(s) as per input filter and returns the *entries* matching the request.
* The returned *entries* contain `instrument identifier` data

The samples below expose several ways to query the reference data service in order to retrieve instrument identifier and fields. 
Please refer to any other scenario in order to discover how instrument identifier is used.

### Services used
This sample uses *gRPC requests* in order to retrieve reference data set of fields from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.static_data.v1*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run symbology queries

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
import google.protobuf as pb
import systemathics.apis.services.static_data.v1.static_data_pb2 as static_data
import systemathics.apis.services.static_data.v1.static_data_pb2_grpc as static_data_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Introduction
The properties one can use as per filter field(s) in order to query the API for equities or futures are the following:
* *Name*, *MIC*, *Ticker*, *Contract code*, *ISIN*, *Cusip*, *market data provider IDs*, etc.

You can combine multiple filters in order to target more accurately your request. Combining filters act as a *AND* operator.

The service returns *entries* matching the request filters.
A reply contains two *arrays* dedicated for *futures* and *equities*. Keep in mind that these arrays might be empty depending on the query filters and the results.
<br>Other shared fields such as <i>`Name`</i> or <i>`Identifier`</i> are there to make it easier to identify the instrument needed (among possible other results).  
Depending on the asset type, there might be additional fields for the aforementioned purpose.

The following code snippets will expose the different ways to query for reference data, with different asset types and various parameters.

#### 3.2 Retrieve reference data - Equities

To request for equities, the user must specify the asset_type as <i>`AssetType.ASSET_TYPE_EQUITY`</i> in the static data request.

In [ ]:
# define a method to handle the equities reponse using a Pandas dataframe
def get_equities_dataframe(response):
    identifier = ['{0}|{1}'.format(equity.identifier.ticker, equity.identifier.exchange) for equity in response.equities]
    type = [equity.type for equity in response.equities]
    country = [equity.country for equity in response.equities]
    name = [equity.name for equity in response.equities]
    currency = [equity.currency for equity in response.equities]
    primary = [equity.primary for equity in response.equities]
    tick_size_rule = [equity.tick_size_rule for equity in response.equities]
    mapping = [get_mapping(equity.mapping) for equity in response.equities]
    index = [equity.index for equity in response.equities]
    open = [equity.open for equity in response.equities]
    close = [equity.close for equity in response.equities]
    time_zone = [equity.time_zone for equity in response.equities]
    lot_size = [equity.lot_size for equity in response.equities]
    point_value = [equity.point_value for equity in response.equities]
    isin = [equity.isin for equity in response.equities]
    cusip = [equity.cusip for equity in response.equities]
    sedol = [equity.sedol for equity in response.equities]
    sectors = [get_sectors(equity.sectors) for equity in response.equities]
    capitalization = [equity.capitalization.value for equity in response.equities]
    
    # Create pandas dataframe
    d = {'Identifier': identifier, 'Type': type, 'Country': country, 'Name': name, 'Currency': currency, 'Primary': primary, 'TickSizeRule': tick_size_rule, 'Mapping':mapping, 'Index': index, 'Open': open, 'Close': close, 'Time zone': time_zone, 'Lot size': lot_size, 'PointValue': point_value, 'Isin': isin, 'Cusip': cusip, 'Sedol': sedol, 'Sectors': sectors, 'Capitalization': capitalization}
    df = pd.DataFrame(data=d)
    return df

In [ ]:
# define methods to handle identifiers mapping and sectors display as a string
def get_mapping(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+'='+value+']'
    return res

def get_sectors(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+','+value+']'
    return res

def get_identifier(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+'='+value+']'
    return res

##### 3.2.1 Equity - by Name

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add name as per filter value
request.name.value = 'Apple'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 3.2.2 Equity - by MIC code

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

request.exchange.value = 'XNGS' # add exchange Code as per filter value
request.count.value = 3 # select the first elements
request.start.value = 2 # starting from element index

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 3.2.3 Equity - by Ticker

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add ticker as per filter value
request.ticker.value = 'AAPL'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 3.2.4 Equity - by Codes

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add isin as per filter value
request.code.value = 'US0378331005'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add bloomberg figi as per filter value
request.code.value = 'BBG000B9Y6P9'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 3.2.5 Equity - by Ticker + MIC code

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add ticker + exchange code as per filter values
request.ticker.value = 'AAPL'
request.exchange.value = 'BATS'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 3.2.6 Equity - index components

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

request.index.value = 'NASDAQ 100' # add ticker + exchange code as per filter values
request.count.value = 10 # select the first elements

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

##### 4.2.7 Equity - sectors

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

# add ticker + exchange code as per filter values
request.equity_sector.value = 'Electronic Computers'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_equities_dataframe(response)
display(data)

#### 3.3 Retrieve reference data - Futures

To request for futures, the user must specify the asset_type as <i>`AssetType.ASSET_TYPE_FUTURE`</i> in the static data request.

In [ ]:
# Define a method to transform the futures in a response into a Pandas dataframe
def get_futures_dataframe(response):
    identifier = ['{0}|{1}'.format(future.identifier.ticker, future.identifier.exchange) for future in response.futures]
    type = [future.type for future in response.futures]
    country = [future.country for future in response.futures]
    name = [future.name for future in response.futures]
    currency = [future.currency for future in response.futures]
    primary = [future.primary for future in response.futures]
    tick_size_rule = [future.tick_size_rule for future in response.futures]
    mapping = [get_mapping(future.mapping) for future in response.futures]
    index = [future.index for future in response.futures]
    open = [future.open for future in response.futures]
    close = [future.close for future in response.futures]
    time_zone = [future.time_zone for future in response.futures]
    lot_size = [future.lot_size for future in response.futures]
    point_value = [future.point_value for future in response.futures]
    underlying = [future.underlying for future in response.futures]
    contract = [future.contract for future in response.futures]
    category = [future.category for future in response.futures]
    chain = [future.chain for future in response.futures]
    maturity = ['{0}-{1:02d}-{2:02d}'.format(future.maturity.year, future.maturity.month,future.maturity.day) for future in response.futures]
    month = [future.month for future in response.futures]
    year = [future.year for future in response.futures]
    
    # Create pandas dataframe
    d = {'Identifier': identifier, 'Type': type, 'Country': country, 'Name': name, 'Currency': currency, 'Primary': primary, 'TickSizeRule': tick_size_rule, 'Mapping':mapping, 'Index': index, 'Open': open, 'Close': close, 'Time zone': time_zone, 'Lot size': lot_size, 'PointValue': point_value, 'Underlying': underlying, 'Contract': contract, 'Category': category, 'Chain': chain, 'Maturity': maturity, 'Month': month, 'Year': year}
    df = pd.DataFrame(data=d)
    return df

##### 3.3.1 Future - by Name

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_FUTURE
)

# add future name as per filter value
request.name.value = 'WTI Crude Futures'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_futures_dataframe(response)
display(data)

##### 3.3.2 Future - by Contract

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_FUTURE
)

# add future contract as per filter value
request.future_contract.value = 'WBS'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_futures_dataframe(response)
display(data)

##### 3.3.3 Future - by Ticker

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_FUTURE
)

# add future ticker as per filter value
request.ticker.value = 'WBSH20'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_futures_dataframe(response)
display(data)

##### 4.3.4 Future - by Category

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_FUTURE
)

# add future ticker as per filter value
request.future_category.value = 'Crude Oil'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_futures_dataframe(response)
display(data)

#### 3.4 Retrieve reference data - ETFs

To request for futures, the user must specify the asset_type as <i>`AssetType.ASSET_TYPE_ETF`</i> in the static data request.

In [ ]:
# Define a method to transform the ETFs in a response into a Pandas dataframe
def get_etfs_dataframe(response):
    identifier = ['{0}|{1}'.format(etf.identifier.ticker, etf.identifier.exchange) for etf in response.etfs]
    type = [etf.type for etf in response.etfs]
    country = [etf.country for etf in response.etfs]
    name = [etf.name for etf in response.etfs]
    currency = [etf.currency for etf in response.etfs]
    primary = [etf.primary for etf in response.etfs]
    tick_size_rule = [etf.tick_size_rule for etf in response.etfs]
    mapping = [get_mapping(etf.mapping) for etf in response.etfs]
    index = [etf.index for etf in response.etfs]
    open = [etf.open for etf in response.etfs]
    close = [etf.close for etf in response.etfs]
    time_zone = [etf.time_zone for etf in response.etfs]
    lot_size = [etf.lot_size for etf in response.etfs]
    point_value = [etf.point_value for etf in response.etfs]
    isin = [etf.isin for etf in response.etfs]
    cusip = [etf.cusip for etf in response.etfs]
    sedol = [etf.sedol for etf in response.etfs]
    sectors = [get_sectors(etf.sectors) for etf in response.etfs]
    capitalization = [etf.capitalization.value for etf in response.etfs]
    
    # Create pandas dataframe
    d = {'Identifier': identifier, 'Type': type, 'Country': country, 'Name': name, 'Currency': currency, 'Primary': primary, 'TickSizeRule': tick_size_rule, 'Mapping':mapping, 'Index': index, 'Open': open, 'Close': close, 'Time zone': time_zone, 'Lot size': lot_size, 'PointValue': point_value, 'Isin': isin, 'Cusip': cusip, 'Sedol': sedol, 'Sectors': sectors, 'Capitalization': capitalization}
    df = pd.DataFrame(data=d)
    return df

##### 3.4.1 ETF - by Name

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_ETF
)

# add name as per filter value
request.name.value = 'Ishares Trust Iboxx Usd High Yield Corporate Bond Etf'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_etfs_dataframe(response)
display(data)

##### 3.4.2 ETF - by MIC code

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_ETF
)

request.exchange.value = 'XNYS' # add exchange Code as per filter value
request.count.value = 3 # select the first elements
request.start.value = 2 # starting from element index

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_etfs_dataframe(response)
display(data)

##### 3.4.3 ETF - by ticker

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_ETF
)

# add ticker as per filter value
request.ticker.value = 'HYG'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_etfs_dataframe(response)
display(data)

##### 3.4.4 Equity - by Ticker + MIC code

In [ ]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_ETF
)

# add ticker + exchange code as per filter values
request.ticker.value = 'IBB'
request.exchange.value = 'XNMS'

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the static data service
        service = static_data_service.StaticDataServiceStub(channel)
        
        # process the request
        response = service.StaticData(request = request, metadata = [('authorization', token)])
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# visualize request results
data = get_etfs_dataframe(response)
display(data)